In [22]:
import pandas as pd
import yfinance as yf
import pandas as pd
import pandas_datareader.data as pdr
from datetime import datetime

start = datetime(2018, 1, 1)
end = datetime(2021, 7, 1)

yf.pdr_override()
stock = pdr.get_data_yahoo('AMZN', start, end)

stock.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1172.000000,1190.000000,1170.510010,1189.010010,1189.010010,2694500
2018-01-03,1188.300049,1205.489990,1188.300049,1204.199951,1204.199951,3108800
2018-01-04,1205.000000,1215.869995,1204.660034,1209.589966,1209.589966,3022100
2018-01-05,1217.510010,1229.140015,1210.000000,1229.140015,1229.140015,3544700
2018-01-08,1236.000000,1253.079956,1232.030029,1246.869995,1246.869995,4279500


In [23]:
signal = pd.DataFrame()

SMA1 = 20
SMA2 = 50
stock['SMA1'] = stock['Close'].rolling(SMA1).mean()
stock['SMA2'] = stock['Close'].rolling(SMA2).mean()

signal['crossover'] = stock['SMA1'] - stock['SMA2']

stock.describe()

signal.tail()

stock = stock.join(signal, how='left')
stock.tail()


,Open,High,Low,Close,Adj Close,Volume,SMA1,SMA2,crossover
Date,,,,,,,,,
2021-06-24,3507.639893,3524.860107,3430.850098,3449.080078,3449.080078,3832000,3340.474500,3320.046797,20.427703
2021-06-25,3464.000000,3464.820068,3394.179932,3401.459961,3401.459961,3941000,3349.041992,3320.494194,28.547798
2021-06-28,3416.000000,3448.000000,3413.510010,3443.889893,3443.889893,2242800,3360.082983,3321.383193,38.699790
2021-06-29,3438.820068,3456.030029,3423.030029,3448.139893,3448.139893,2098400,3371.557483,3322.905791,48.651692
2021-06-30,3441.060059,3471.600098,3435.000000,3440.159912,3440.159912,2404000,3381.865979,3325.015190,56.850789


In [24]:
signal['close-close'] = stock['Close'].diff()
signal.head()

,crossover,close-close
Date,,
2018-01-02,NaN,NaN
2018-01-03,NaN,15.189941
2018-01-04,NaN,5.390015
2018-01-05,NaN,19.550049
2018-01-08,NaN,17.729980


In [26]:
signal.dropna(inplace=True)
quality = (signal['crossover'] > 0) & (signal['close-close'] > 0) | (signal['crossover'] < 0) & (signal['close-close'] < 0)
quality.head()

Date
2018-03-14     True
2018-03-15    False
2018-03-16    False
2018-03-19    False
2018-03-20     True
dtype: bool

In [27]:
quality = quality.astype(int)
quality.head()

Date
2018-03-14    1
2018-03-15    0
2018-03-16    0
2018-03-19    0
2018-03-20    1
dtype: int32

In [28]:
proportion_correct = quality.sum()/quality.count()
proportion_correct

0.50661853188929

In [29]:
information_coefficient = 2 * proportion_correct - 1
information_coefficient

0.013237063778579916

In [30]:
signal.corr(method='spearman')

,crossover,close-close
crossover,1.000000,-0.027932
close-close,-0.027932,1.000000
